In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as dset
import torchvision.transforms as T
import matplotlib.pyplot as plt
import random
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler
from builtins import range
from builtins import object
import numpy as np
from past.builtins import xrange
import math
from scipy.spatial import distance

!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
import torchvision.datasets as dset
from torchvision import transforms, utils, datasets
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler, sampler

from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
class KNearestNeighbor(object):
    """ a kNN classifier with L2 distance """

    def __init__(self):
        pass

    def train(self, X, y):
        """
        Train the classifier. For k-nearest neighbors this is just
        memorizing the training data.

        Inputs:
        - X: A numpy array of shape (num_train, D) containing the training data
          consisting of num_train samples each of dimension D.
        - y: A numpy array of shape (N,) containing the training labels, where
             y[i] is the label for X[i].
        """
        self.X_train = X
        self.y_train = y

    def predict(self, X, k, num_loops=0):
        """
        Predict labels for test data using this classifier.

        Inputs:
        - X: A numpy array of shape (num_test, D) containing test data consisting
             of num_test samples each of dimension D.
        - k: The number of nearest neighbors that vote for the predicted labels.
        - num_loops: Determines which implementation to use to compute distances
          between training points and testing points.

        Returns:
        - y: A numpy array of shape (num_test,) containing predicted labels for the
          test data, where y[i] is the predicted label for the test point X[i].
        """
        if num_loops == 0:
            dists = self.compute_distances_no_loops(X)
        elif num_loops == 1:
            dists = self.compute_distances_one_loop(X)
        elif num_loops == 2:
            dists = self.compute_distances_two_loops(X)
        else:
            raise ValueError('Invalid value %d for num_loops' % num_loops)

        return self.predict_labels(dists, k=k)

    def compute_distances_no_loops(self, X):
        """
        Compute the distance between each test point in X and each training point
        in self.X_train using no explicit loops.

        Input / Output: Same as compute_distances_two_loops
        """
        num_test = X.shape[0]
        num_train = self.X_train.shape[0]
        dists = np.zeros((num_test, num_train))
        w = (X**2).sum(axis=1)[:, np.newaxis]
        x = (self.X_train**2).sum(axis=1)
        y = 2 * X.dot(self.X_train.T)
        z = w + x - y
        dists = np.sqrt(z)
        return dists

    def predict_labels(self, dists, k):
        """
        Given a matrix of distances between test points and training points,
        predict a label for each test point.

        Inputs:
        - dists: A numpy array of shape (num_test, num_train) where dists[i, j]
          gives the distance betwen the ith test point and the jth training point.

        Returns:
        - y: A numpy array of shape (num_test,) containing predicted labels for the
          test data, where y[i] is the predicted label for the test point X[i].
        """
        num_test = dists.shape[0]
        y_pred = np.zeros(num_test)
        for i in range(num_test):
            # A list of length k storing the labels of the k nearest neighbors to
            # the ith test point.
            closest_y = []
            arr = np.argsort(dists[i])
            for j in range(k):
              closest_y.append(self.y_train[arr[j]])
            y_pred[i] = max(set(closest_y), key=closest_y.count)

        return y_pred


In [5]:
torch.manual_seed(0)
%cd drive/My\ Drive/final_project
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("We're using =>", device)
!pwd
root_dir = "."

/content/drive/My Drive/final_project
We're using => cuda
/content/drive/My Drive/final_project
The data lies here => .


In [20]:

image_transforms = {
    "train": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
}

train = datasets.ImageFolder(root ="./data-new",transform = image_transforms["train"])
train_indices = list(range(len(train)))
np.random.shuffle(train_indices)
train_indices = train_indices[:5000] # remove later
val_split_index = int(np.floor(0.2 * 5000)) # change to len train later
train_idx, val_idx = train_indices[val_split_index:], train_indices[:val_split_index]
print(train_idx, val_idx)
print(len(train_idx), len(val_idx))
train_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(val_idx)

train_loader = DataLoader(dataset=train, shuffle=False, batch_size=len(train), sampler=train_sampler)
val_loader = DataLoader(dataset=train, shuffle=False, batch_size=len(train), sampler=val_sampler)
# test_loader = DataLoader(dataset=test, shuffle=False, batch_size=1)

[3742, 2494, 9367, 16526, 17049, 12950, 7976, 11036, 18294, 20661, 5275, 4182, 934, 1165, 1592, 3885, 9846, 15083, 6347, 3221, 13046, 3037, 19561, 15150, 14893, 13059, 3934, 803, 5750, 1325, 4031, 20820, 6260, 5680, 21295, 14468, 16368, 16169, 11332, 5428, 20460, 1632, 12460, 12955, 16676, 6329, 3364, 9731, 14831, 12269, 12687, 6474, 10145, 19832, 5592, 1651, 853, 12668, 14868, 3804, 14984, 4828, 3776, 3896, 6446, 868, 4997, 14030, 4033, 10504, 1053, 19811, 77, 16468, 17005, 12102, 8265, 21316, 12915, 6952, 17224, 15226, 7893, 13800, 7068, 19666, 17638, 19228, 13855, 15841, 15734, 17822, 17890, 13441, 8182, 6676, 12751, 1163, 1916, 14502, 9909, 314, 9961, 18729, 3377, 15073, 6861, 16122, 124, 3085, 15916, 16558, 10218, 9141, 7690, 1793, 8799, 10948, 18250, 3067, 7550, 14631, 13032, 16938, 9271, 21022, 659, 821, 5741, 13026, 17682, 10050, 2191, 8057, 5371, 14787, 5633, 17648, 8109, 5738, 11472, 6563, 20207, 17227, 15480, 10708, 5280, 7149, 7029, 12822, 20955, 18775, 18275, 9212, 114, 81

In [22]:
for batch_idx, (X_train, y_train) in enumerate(train_loader):
    print("done loading dataset")
    X_train = X_train.numpy()
    y_train = y_train.numpy()
    X_train = np.reshape(X_train, (X_train.shape[0], -1))
    print('Training data shape: ', X_train.shape)
    print('Training labels shape: ', y_train.shape)
    classifier = KNearestNeighbor()
    classifier.train(X_train, y_train)
    print(y_train)
  
for batch_idx, (X_test, y_test) in enumerate(val_loader):
    X_test = X_test.numpy()
    y_test = y_test.numpy()
    X_test = np.reshape(X_test, (X_test.shape[0], -1))
    print('Testing data shape: ', X_test.shape)
    print('Testing labels shape: ', y_test.shape)
    print(y_test)
    dists = classifier.compute_distances_no_loops(X_test)
    # print(dists)
    y_test_pred = classifier.predict_labels(dists, k=5)
    num_correct = np.sum(y_test_pred == y_test)
    accuracy = float(num_correct) / y_test.shape[0]
    print('Got %d / %d correct => accuracy: %f' % (num_correct, y_test.shape[0], accuracy))
  

done loading dataset
Training data shape:  (4000, 150528)
Training labels shape:  (4000,)
[0 1 1 ... 1 0 1]
Testing data shape:  (1000, 150528)
Testing labels shape:  (1000,)
[0 1 1 1 0 0 0 1 1 0 1 1 1 1 0 1 0 1 1 1 1 0 1 1 1 1 0 0 1 0 0 1 0 0 1 1 1
 1 1 1 0 1 0 1 1 0 1 0 1 1 0 1 0 1 1 0 0 0 1 1 0 1 1 1 0 0 0 0 1 1 1 1 0 0
 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 0 0 1 0 0 0 1 1 1 1 0
 1 1 0 0 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 0 0 1 1 0 1 0 1 1 1 1 1 0 0 0 1 1 1
 0 0 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 0 1 0 1 0 0 0 1 1 1 1 0 0 1 1 1 1 1
 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 0 1 0 0 1 1 1 1 1 1
 0 0 1 0 1 1 1 0 1 0 1 1 1 1 0 0 1 0 1 0 0 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1
 1 1 1 1 1 0 1 1 0 1 0 1 0 1 1 0 1 1 0 1 1 0 0 1 1 1 0 1 1 0 1 1 1 1 1 0 0
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 1 0 1 1 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1
 1 1 0 0 1 0 0 0 0 0 1 1 0 1 1 1 1 0 1 0 0 1 1 0 1 1 1 1 1 0 0 0 0 1 1 1 0
 1 1 1 0 0 0 1 0 1 1 0 0 1 0 1 0 1 1 1 1 0 1 1 0 1 0 0 1 1 1 0 1 1 0 0 1 1
